In [ ]:
import asyncio
import sys
import logging
import boto3
from boto3.session import Session
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from botocore.auth import SigV4Auth

from collections.abc import AsyncGenerator
from contextlib import asynccontextmanager
from datetime import timedelta
from typing import Generator

import httpx
from anyio.streams.memory import MemoryObjectReceiveStream, MemoryObjectSendStream
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from botocore.credentials import Credentials
from mcp.client.streamable_http import (
    GetSessionIdCallback,
    StreamableHTTPTransport,
    streamablehttp_client,
)
from mcp.shared._httpx_utils import McpHttpClientFactory, create_mcp_http_client
from mcp.shared.message import SessionMessage
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
import json
import os
from typing import Any
from contextlib import asynccontextmanager
from langchain_core.tools import tool



In [2]:
class SigV4HTTPXAuth(httpx.Auth):
    """HTTPX Auth class that signs requests with AWS SigV4."""

    def __init__(
        self,
        credentials: Credentials,
        service: str,
        region: str,
    ):
        self.credentials = credentials
        self.service = service
        self.region = region
        self.signer = SigV4Auth(credentials, service, region)

    def auth_flow(
        self, request: httpx.Request
    ) -> Generator[httpx.Request, httpx.Response, None]:
        """Signs the request with SigV4 and adds the signature to the request headers."""

        # Create an AWS request
        headers = dict(request.headers)
        # Header 'connection' = 'keep-alive' is not used in calculating the request
        # signature on the server-side, and results in a signature mismatch if included
        headers.pop("connection", None)  # Remove if present, ignore if not

        aws_request = AWSRequest(
            method=request.method,
            url=str(request.url),
            data=request.content,
            headers=headers,
        )

        # Sign the request with SigV4
        self.signer.add_auth(aws_request)

        # Add the signature header to the original request
        request.headers.update(dict(aws_request.headers))

        yield request

In [3]:
class StreamableHTTPTransportWithSigV4(StreamableHTTPTransport):
    """
    Streamable HTTP client transport with AWS SigV4 signing support.

    This transport enables communication with MCP servers that authenticate using AWS IAM,
    such as servers behind a Lambda function URL or API Gateway.
    """

    def __init__(
        self,
        url: str,
        credentials: Credentials,
        service: str,
        region: str,
        headers: dict[str, str] | None = None,
        timeout: float | timedelta = 30,
        sse_read_timeout: float | timedelta = 60 * 5,
    ) -> None:
        """Initialize the StreamableHTTP transport with SigV4 signing.

        Args:
            url: The endpoint URL.
            credentials: AWS credentials for signing.
            service: AWS service name (e.g., 'lambda').
            region: AWS region (e.g., 'us-east-1').
            headers: Optional headers to include in requests.
            timeout: HTTP timeout for regular operations.
            sse_read_timeout: Timeout for SSE read operations.
        """
        # Initialize parent class with SigV4 auth handler
        super().__init__(
            url=url,
            headers=headers,
            timeout=timeout,
            sse_read_timeout=sse_read_timeout,
            auth=SigV4HTTPXAuth(credentials, service, region),
        )

        self.credentials = credentials
        self.service = service
        self.region = region

In [4]:

@asynccontextmanager
async def streamablehttp_client_with_sigv4(
    url: str,
    credentials: Credentials,
    service: str,
    region: str,
    headers: dict[str, str] | None = None,
    timeout: float | timedelta = 30,
    sse_read_timeout: float | timedelta = 60 * 5,
    terminate_on_close: bool = True,
    httpx_client_factory: McpHttpClientFactory = create_mcp_http_client,
) -> AsyncGenerator[
    tuple[
        MemoryObjectReceiveStream[SessionMessage | Exception],
        MemoryObjectSendStream[SessionMessage],
        GetSessionIdCallback,
    ],
    None,
]:
    """
    Client transport for Streamable HTTP with SigV4 auth.

    This transport enables communication with MCP servers that authenticate using AWS IAM,
    such as servers behind a Lambda function URL or API Gateway.

    Yields:
        Tuple containing:
            - read_stream: Stream for reading messages from the server
            - write_stream: Stream for sending messages to the server
            - get_session_id_callback: Function to retrieve the current session ID
    """

    async with streamablehttp_client(
        url=url,
        headers=headers,
        timeout=timeout,
        sse_read_timeout=sse_read_timeout,
        terminate_on_close=terminate_on_close,
        httpx_client_factory=httpx_client_factory,
        auth=SigV4HTTPXAuth(credentials, service, region),
    ) as result:
        yield result

In [5]:
def create_streamable_http_transport_sigv4(mcp_url: str, service_name: str, region: str):
    session = boto3.Session()
    credentials = session.get_credentials()
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )

In [6]:
async def llm_mcp_handler(mcp_session, region, query):
    try:
        # MCP 도구 정보 가져오기
        mcp_tools = await mcp_session.list_tools()
        tools_list = mcp_tools.tools if hasattr(mcp_tools, 'tools') else mcp_tools
        
        # LLM 초기화
        bedrock_client = boto3.client('bedrock-runtime', region_name=region)
        llm = ChatBedrock(
            client=bedrock_client,
            model_id="anthropic.claude-3-sonnet-20240229-v1:0",
            model_kwargs={"max_tokens": 1000, "temperature": 0}
        )
        
        # 도구 정보를 스키마와 함께 텍스트로 변환
        tool_descriptions = []
        for tool_info in tools_list:
            schema = tool_info.inputSchema
            required_params = schema.get('required', [])
            properties = schema.get('properties', {})
            
            param_info = []
            for param in required_params:
                param_type = properties.get(param, {}).get('type', 'unknown')
                param_info.append(f"{param} ({param_type})")
            
            tool_descriptions.append(
                f"- {tool_info.name}: {tool_info.description}\n"
                f"  필수 파라미터: {', '.join(param_info)}"
            )
        
        tools_text = "\n".join(tool_descriptions)
        
        # 개선된 도구 선택 프롬프트
        selection_prompt = f"""
        사용 가능한 도구들:
        {tools_text}
        
        질문: {query}
        
        이 질문을 분석하세요:
        1. 수학 계산이 필요한가? → add_numbers 또는 multiply_numbers 사용
        2. 사용자 인사가 필요한가? → greet_user 사용  
        3. 위 도구들로 해결할 수 없는 일반적인 질문인가? → "none" 선택
        
        응답 형식:
        {{
            "tool": "도구명 또는 none",
            "params": {{"파라미터": 값}} 또는 {{}}
        }}
        
        JSON만 응답하세요:
        """
        
        # LLM으로 도구 선택 및 파라미터 생성
        response = await llm.ainvoke(selection_prompt)
        
        try:
            import json
            decision = json.loads(response.content.strip())
            
            tool_name = decision.get("tool")
            params = decision.get("params", {})
            
            print(f"Selected tool: {tool_name}, params: {params}")
            
            if tool_name and tool_name.lower() != "none":
                # MCP 도구 실행
                tool_result = await mcp_session.call_tool(tool_name, params)
                return f"🔧 도구 '{tool_name}' 결과: {tool_result}"
            else:
                # 도구 없이 직접 답변
                direct_response = await llm.ainvoke(f"질문에 답하세요: {query}")
                return f"🤖 직접 답변: {direct_response.content}"
                
        except (json.JSONDecodeError, KeyError) as e:
            print(f"JSON 파싱 오류: {e}")
            # JSON 파싱 실패 시 직접 답변
            direct_response = await llm.ainvoke(f"질문에 답하세요: {query}")
            return f"🤖 직접 답변: {direct_response.content}"
        
    except Exception as e:
        return f"❌ Error: {str(e)}"



In [7]:
async def extract_text(payload):
    try:
        yield {"type": "status", "message": "🚀 Initializing LLM..."}
        
        boto_session = Session()
        region = boto_session.region_name or 'us-west-2'
        ssm_client = boto3.client("ssm", region_name=region)
        
        agent_arn_response = ssm_client.get_parameter(Name="/mcp_server/runtime_iam/agent_arn")
        agent_arn = agent_arn_response["Parameter"]["Value"]
        
        encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
        mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
        
        yield {"type": "status", "message": "✅ Connecting to MCP..."}
        
        stderr_backup = sys.stderr
        sys.stderr = open(os.devnull, 'w')
        
        try:
            async with create_streamable_http_transport_sigv4(
                mcp_url=mcp_url, service_name="bedrock-agentcore", region=region
            ) as (read_stream, write_stream, _):
                async with ClientSession(read_stream, write_stream) as mcp_session:
                    await mcp_session.initialize()
                    
                    yield {"type": "status", "message": "✅ Processing..."}
                    
                    response = await llm_mcp_handler(mcp_session, region, payload["input_data"])
                    yield {"type": "stream", "content": response}
        finally:
            sys.stderr.close()
            sys.stderr = stderr_backup
                
    except Exception as e:
        yield {"type": "error", "message": str(e)}

In [8]:
async def run_in_mpc_agentic_test():
    payload = {"input_data": "4+4??"}
    
    async for result in extract_text(payload):
        if result["type"] == "status":
            print(f"Status: {result['message']}")
        elif result["type"] == "stream":
            print(f"Response: {result['content']}")
        elif result["type"] == "error":
            print(f"Error: {result['message']}")



In [9]:
await run_in_mpc_agentic_test()


Status: 🚀 Initializing LLM...
Status: ✅ Connecting to MCP...
Status: ✅ Processing...
Selected tool: add_numbers, params: {'a': 4, 'b': 4}
Response: 🔧 도구 'add_numbers' 결과: meta=None content=[TextContent(type='text', text='8', annotations=None, meta=None)] structuredContent={'result': 8} isError=False


In [10]:
async def run_in_mpc_agentic_test():
    payload = {"input_data": "달의 온도는??"}
    
    async for result in extract_text(payload):
        if result["type"] == "status":
            print(f"Status: {result['message']}")
        elif result["type"] == "stream":
            print(f"Response: {result['content']}")
        elif result["type"] == "error":
            print(f"Error: {result['message']}")


In [11]:
await run_in_mpc_agentic_test()


Status: 🚀 Initializing LLM...
Status: ✅ Connecting to MCP...
Status: ✅ Processing...
Selected tool: none, params: {}
Response: 🤖 직접 답변: 달의 온도는 매우 극심한 변화를 보입니다. 이는 달에 대기층이 없어 태양열을 저장하거나 방출할 수 없기 때문입니다.

- 낮 동안 달 표면의 최고 온도는 약 127°C(261°F)까지 올라갑니다.
- 밤이 되면 온도가 급격히 떨어져 약 -173°C(-279°F)까지 내려갑니다.

이렇게 극심한 온도 변화는 하루 동안 약 300°C(540°F) 이상의 차이를 보입니다. 달 표면의 평균 온도는 약 -23°C(-9°F)입니다. 이처럼 달에는 대기가 없어 온도 조절이 어려운 환경입니다.


In [12]:
async def run_in_mpc_agentic_test():
    payload = {"input_data": "2*2??"}
    
    async for result in extract_text(payload):
        if result["type"] == "status":
            print(f"Status: {result['message']}")
        elif result["type"] == "stream":
            print(f"Response: {result['content']}")
        elif result["type"] == "error":
            print(f"Error: {result['message']}")


In [13]:
await run_in_mpc_agentic_test()


Status: 🚀 Initializing LLM...
Status: ✅ Connecting to MCP...
Status: ✅ Processing...
Selected tool: multiply_numbers, params: {'a': 2, 'b': 2}
Response: 🔧 도구 'multiply_numbers' 결과: meta=None content=[TextContent(type='text', text='4', annotations=None, meta=None)] structuredContent={'result': 4} isError=False
